<a href="https://colab.research.google.com/github/tejpat98/Textbook-Summarisation/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preprocessing textbook dataset:**
*   Download textbooks (OP / CC licences)
*   Convert PDF into raw text data
*   Split textbooks into chapters
*   Put into a labeled dataset format


Notes:
https://stackoverflow.com/questions/60754884/python-ocr-pytesseract-for-pdf 


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract

In [ ]:
!add-apt-repository ppa:alex-p/tesseract-ocr
!apt-get update
!apt install libtesseract-dev

In [ ]:
!nvidia-smi

Wed Jan 27 14:34:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for i, gpu in enumerate(GPUs):
  print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))


In [ ]:
import os
from PIL import Image
from pdf2image import convert_from_path
import pytesseract

Convert PDF into Images

In [ ]:
%cd '/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/27/'
path='/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/27/images/'
fileName = '27.pdf'
doc = convert_from_path(fileName, 300, output_folder=path, paths_only=True, fmt="jpeg")


Convert Images into Text using PyTesseract

In [ ]:
for i in range(23,28):
  path = '/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/'+str(i)+'/images/'
  ImageNames = os.listdir("/content/drive/MyDrive/University/Computer Science - Level 3/FYP/Open and CC Textbooks (PDF)/"+str(i)+"/images/")
  print(len(ImageNames))
  PrefList= ImageNames[0].strip().split('-')[:-1]
  pref = '-'.join(PrefList)

  for i in range(1,len(ImageNames)):
    with open(path+'**output.txt', 'a') as f:
      txt = pytesseract.image_to_string(Image.open(path+str(pref)+'-'+str(i).zfill(len(str(len(ImageNames))))+'.jpg'))
      f.write("*****Page: " + str(i) +"*****")
      f.write("\n")
      f.write(txt)

1. Clean textbook dataset
2. Create extractive summaries(simple ones) for the textbook dataset
3. Augment with whole [science_papers](https://www.tensorflow.org/datasets/catalog/scientific_papers) dataset from TF
4. Train a basic seq2seq model on 80% + science_papers datasets
5. Test model using 20% of textbook dataset

In [ ]:
import json
counter = 1
offset = 9
Book = {
    'chapter_name': [],
    'chapter_text': [],
    'cleaned_chapter_text': []
}
chapter = [
    [""],
    ["EOF", 153]]
i = 0
chapter_text = []
with open('C:/Users/Tejash/Desktop/Comp Sci/Year 3/FYP/Textbook dataset/__output (7).txt', encoding='utf-8', buffering=1) as f:
    for line in f:
        if "*****Page: " in line:
            #Found start of new page
            pageNum = line.split(" ")[1][:-6]

        if int(pageNum) >= (chapter[i][1] + offset) and int(pageNum) < (chapter[i+1][1] + offset):
            chapter_text.append(str(line))
        elif int(pageNum) == (chapter[i+1][1] + offset):
            Book['chapter_name'].append(chapter[i][0])
            Book['chapter_text'].append(chapter_text[:])
            chapter_text.clear()
            i +=1

Book['chapter_name'].append(chapter[i][0])
Book['chapter_text'].append(chapter_text[:])
chapter_text.clear()

dump = json.dumps(Book)
with open('C:/Users/Tejash/Desktop/Comp Sci/Year 3/FYP/Textbook dataset/TB json/book_7.txt', 'x') as f2:
    f2.write(dump)
    f2.close()

In [ ]:
import re
import json

# 1. Remove bad chapters
# 2. Remove empty string, page number headers, punctuation, all numbers, URLs and special characters/symbols (including " \n ")
# 3. Put all strings into a spell checker
# 4. ONLY then do stemming and other NLP optimisations
Book = {
    'chapter_name': [],
    'chapter_text': [],
    'cleaned_chapter_text': []
}


def checkResult(title):
    index = 0
    print(title + ": ")
    for m, n in enumerate(Book['cleaned_chapter_text']):
        if m == index:
            print(str(m) + ": ", n)
    return


def decontracte(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    return phrase


with open('C:/Users/Tejash/Desktop/Comp Sci/Year 3/FYP/Textbook dataset/TB json/book_7.txt', buffering=1) as f:
    for line in f:
        data = json.loads(line)
        for i, j in enumerate(data['chapter_name']):
            Book['chapter_name'].append(data['chapter_name'][i])
            Book['chapter_text'].append(data['chapter_text'][i])
            Book['cleaned_chapter_text'].append(data['chapter_text'][i])

checkResult("Original")

# Remove new lines "\n"
for i, j in enumerate(Book['cleaned_chapter_text']):
    for k, l in enumerate(j):
        Book['cleaned_chapter_text'][i][k] = Book['cleaned_chapter_text'][i][k].replace(" \n ", "")
        Book['cleaned_chapter_text'][i][k] = Book['cleaned_chapter_text'][i][k].replace(" \n", "")
        Book['cleaned_chapter_text'][i][k] = Book['cleaned_chapter_text'][i][k].replace("\n ", "")
        Book['cleaned_chapter_text'][i][k] = Book['cleaned_chapter_text'][i][k].replace("\n", "")
checkResult("Removed newline")

# Remove empty string
for x in range(2):
    for i2, j2 in enumerate(Book['cleaned_chapter_text']):
        for k2, l2 in enumerate(j2):
            if l2 == "":
                del Book['cleaned_chapter_text'][i2][k2]
checkResult("Removed empty strings")

# Remove page tags
for i3, j3 in enumerate(Book['cleaned_chapter_text']):
    for k3, l3 in enumerate(j3):
        if l3.find("*****Page: ") != -1:
            del Book['cleaned_chapter_text'][i3][k3]
checkResult("Removed page tags")

# Remove URL
for i4, j4 in enumerate(Book['cleaned_chapter_text']):
    for k4, l4 in enumerate(j4):
        URL = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', l4)
        if URL:
            Book['cleaned_chapter_text'][i4][k4] = Book['cleaned_chapter_text'][i4][k4].replace(URL[0], "@xURL")
checkResult("Removed URLs")

# Remove numbers
for i5, j5 in enumerate(Book['cleaned_chapter_text']):
    for k5, l5 in enumerate(j5):
        no_nums = re.sub(r'\d+', '', l5)
        Book['cleaned_chapter_text'][i5][k5] = no_nums
checkResult("Removed numbers")

# Remove contractions
for i6, j6 in enumerate(Book['cleaned_chapter_text']):
    for k6, l6 in enumerate(j6):
        no_cont = decontracte(str(l6))
        Book['cleaned_chapter_text'][i6][k6] = no_cont
checkResult("Removed contractions")

# remove punctuation
for i7, j7 in enumerate(Book['cleaned_chapter_text']):
    for k7, l7 in enumerate(j7):
        no_punc = re.sub(r'[^\w]', ' ', l7)
        Book['cleaned_chapter_text'][i7][k7] = no_punc
checkResult("Remove punctuation")

dump = json.dumps(Book)
with open('C:/Users/Tejash/Desktop/Comp Sci/Year 3/FYP/Textbook dataset/TB json/with cleaned/book_cleaned_9999.txt', 'x') as f2:
    f2.write(dump)
    f2.close()


Preprocessing research papers

In [ ]:
import re
import json

papers = {
    'article_id': [],
    'article_text': [],
    'cleaned_article_text': [],
    'abstract_text': []}


def checkResult(title):
    index = 0
    print(title + ": ")
    print(papers['cleaned_article_text'][0][0])
    return


def decontracte(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    # general
    phrase = re.sub(r"non-", "not ", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    return phrase


with open("C:/Users/Tejash/Desktop/Comp Sci/Year 3/FYP/arxiv-dataset/arxiv-dataset/train_split/train_split_21_3037.txt", encoding='utf-8', buffering=1) as f:
    for line in f:
        data = json.loads(line)
        papers['article_id'].append(data['article_id'])
        papers['article_text'].append(data['article_text'])
        papers['cleaned_article_text'].append(data['article_text'])
        papers['abstract_text'].append(data['abstract_text'])

checkResult("Original")

# Remove new lines "\n"
for i, j in enumerate(papers['article_id']):
    for k, l in enumerate(papers['cleaned_article_text'][i]):
        for m, n in enumerate(l):
            papers['cleaned_article_text'][i][k][m] = papers['cleaned_article_text'][i][k][m].replace(" \n ", "")
            papers['cleaned_article_text'][i][k][m] = papers['cleaned_article_text'][i][k][m].replace(" \n", "")
            papers['cleaned_article_text'][i][k][m] = papers['cleaned_article_text'][i][k][m].replace("\n ", "")
            papers['cleaned_article_text'][i][k][m] = papers['cleaned_article_text'][i][k][m].replace("\n", "")

checkResult("Removed newline")

# Remove empty string
for x in range(2):
    for i2, j2 in enumerate(papers['article_id']):
        for k2, l2 in enumerate(papers['cleaned_article_text'][i2]):
            for m2, n2 in enumerate(l2):
                if n2 == "":
                    del papers['cleaned_article_text'][i2][k2][m2]
checkResult("Removed empty strings")

# Remove numbers
for i4, j4 in enumerate(papers['article_id']):
    for k4, l4 in enumerate(papers['cleaned_article_text'][i4]):
        for m4, n4 in enumerate(l4):
            no_nums = re.sub(r'\d+', '', n4)
            papers['cleaned_article_text'][i4][k4][m4] = no_nums
checkResult("Removed numbers")

# Remove contractions
for i5, j5 in enumerate(papers['article_id']):
    for k5, l5 in enumerate(papers['cleaned_article_text'][i5]):
        for m5, n5 in enumerate(l5):
            no_cont = decontracte(str(n5))
            papers['cleaned_article_text'][i5][k5][m5] = no_cont
checkResult("Removed contractions")

# remove punctuation
for i6, j6 in enumerate(papers['article_id']):
    for k6, l6 in enumerate(papers['cleaned_article_text'][i6]):
        for m6, n6 in enumerate(l6):
            no_punc = re.sub(r'[^\w]', ' ', n6)
            papers['cleaned_article_text'][i6][k6][m6] = no_punc
checkResult("Remove punctuation")

dump = json.dumps(papers)
with open('C:/Users/Tejash/Desktop/Comp Sci/Year 3/FYP/Textbook dataset/TB json/cleaned papers/paper_cleaned_1.txt',
          'x') as f2:
    f2.write(dump)
    f2.close()
